# Sagemaker con Tensorflow: IMDB.

In [ ]:
!pip install -q sagemaker-experiments

In [ ]:
import sagemaker
import json
import boto3

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name
bucket = sess.default_bucket()
prefix = 'module_4/part_9'

In [ ]:
import numpy as np
import os

In [ ]:
import tensorflow as tf
from tensorflow.python.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

#### Preparación de los datos
- Base de datos de opiniones de IMDB
- https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb/load_data

In [ ]:
max_features = 20000
maxlen = 400

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

In [ ]:
x_train

In [ ]:
data_dir = os.path.join(os.getcwd(), 'imdb_data')
os.makedirs(data_dir, exist_ok=True)

train_dir = os.path.join(os.getcwd(), 'imdb_data/train')
os.makedirs(train_dir, exist_ok=True)

test_dir = os.path.join(os.getcwd(), 'imdb_data/test')
os.makedirs(test_dir, exist_ok=True)

csv_test_dir = os.path.join(os.getcwd(), 'imdb_data/csv-test')
os.makedirs(csv_test_dir, exist_ok=True)

np.save(os.path.join(train_dir, 'x_train.npy'), x_train)
np.save(os.path.join(train_dir, 'y_train.npy'), y_train)
np.save(os.path.join(test_dir, 'x_test.npy'), x_test)
np.save(os.path.join(test_dir, 'y_test.npy'), y_test)
np.savetxt(os.path.join(csv_test_dir, 'csv-test.csv'), 
           np.array(x_test[:100], dtype=np.int32), fmt='%d', delimiter=",")

In [ ]:
traindata_s3_prefix = f'{prefix}/imdb_data/train'
testdata_s3_prefix = f'{prefix}/imdb_data/test'

train_s3 = sess.upload_data(path='./imdb_data/train/', key_prefix=traindata_s3_prefix)
test_s3 = sess.upload_data(path='./imdb_data/test/', key_prefix=testdata_s3_prefix)

## Entrenamiento del Modelo

In [ ]:
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from botocore.exceptions import ClientError
from time import gmtime, strftime
import time

experiment_name = 'imdb-sentiment-analysis'

try:
    experiment = Experiment.create(
        experiment_name=experiment_name, 
        description='Training a sentiment classification model using imdb dataset.')
except ClientError as e:
    print(f'{experiment_name} experiment already exists! Reusing the existing experiment.')
    

In [ ]:
from sagemaker.tensorflow import TensorFlow

exp_datetime = strftime('%Y-%m-%d-%H-%M-%S', gmtime())
jobname = f'imdb-tf-{exp_datetime}'

s3_output_location = f's3://{bucket}/{prefix}/{jobname}'
code_dir = f's3://{bucket}/{prefix}/{jobname}'

train_instance_type = 'ml.p3.2xlarge'

hyperparameters = {
    'epochs': 10,
    'batch_size': 64, 
    'learning_rate': 0.01,
    'drop_out_rate': 0.2
}

model_local_path = '/opt/ml/model'

estimator = TensorFlow(
    source_dir='tf_code',
    entry_point='tensorflow_sentiment.py',
    output_path=s3_output_location,
    code_location=code_dir,
    model_dir=model_local_path,
    instance_type=train_instance_type,
    instance_count=1,
    enable_sagemaker_metrics=True,
    hyperparameters=hyperparameters,
    role=role,
    framework_version='2.1',
    py_version='py3'
)

data_channels = {'train':train_s3, 'test': test_s3}
print(data_channels)

In [ ]:
# Creating a new trial for the experiment
exp_trial = Trial.create(experiment_name=experiment_name, 
                         trial_name=jobname)

experiment_config={'ExperimentName': experiment_name,
                   'TrialName': exp_trial.trial_name,
                   'TrialComponentDisplayName': 'Training'}

estimator.fit(inputs=data_channels,
              job_name=jobname,
              experiment_config=experiment_config,
              logs=True)

#### Prueba del modelo en local

In [ ]:
!mkdir ./imdb_data/model -p
!aws s3 cp {estimator.model_data} ./imdb_data/model.tar.gz
!tar -xzf ./imdb_data/model.tar.gz -C ./imdb_data/model/

In [ ]:
my_model=tf.keras.models.load_model('./imdb_data/model/1/')

In [ ]:
my_model.summary()

In [ ]:
loss, acc=my_model.evaluate(x_test, y_test, verbose=2)
print(f'Restored model, accuracy: {100 * acc}%')